In [1]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker

import mpl_toolkits
from mpl_toolkits import basemap
import matplotlib.cm as cm

import numba

mp.rcParams.update({'mathtext.default': 'regular'})
% matplotlib inline

In [2]:
working_dir = '/Users/baird/google_drive/_data_original/NCAR_LENS/daily/PRECT/B1850C5CN_CA_REGION/'

In [3]:
file_list = numpy.array(( \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.04020101-04991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.05000101-05991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.06000101-06991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.07000101-07991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.08000101-08991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.09000101-09991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.10000101-10991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.11000101-11991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.12000101-12991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.13000101-13991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.14000101-14991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.15000101-15991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.16000101-16991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.17000101-17991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.18000101-18991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.19000101-19991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.20000101-20991231_CA_REGION.nc', \
'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.21000101-22001231_CA_REGION.nc' ))

In [4]:
d_oct = 31
d_nov = 30
d_dec = 31
d_jan = 31
d_feb = 28
d_mar = 31
days_per_season = d_oct+d_nov+d_dec+d_jan+d_feb+d_mar

In [7]:
i = 0

ncfile = netCDF4.Dataset(working_dir + file_list[i])
PRECT_lat = ncfile.variables['lat'][:]
PRECT_lon = ncfile.variables['lon'][:]
PRECT_time_var = ncfile.variables['time']

PRECT_time_dates = netCDF4.num2date(PRECT_time_var[:], PRECT_time_var.units, PRECT_time_var.calendar)
time_indices_NDJFM = numpy.array([t.month in [11,12,1,2,3] for t in PRECT_time_dates], dtype=bool)
PRECT_time_dates_NDJFM = PRECT_time_dates[time_indices_NDJFM]

#LA_lat_idx = geo_idx(LA_lat, PRECT_lat)
#LA_lon_idx = geo_idx(LA_lon, PRECT_lon)

#PRECT_ONDJFM_CA = ncfile.variables['PRECT'][time_indices_ONDJFM,(LA_lat_idx-2):(LA_lat_idx+2), (LA_lon_idx-2):(LA_lon_idx+2)]*86400.*1000
PRECT_NDJFM_CA = ncfile.variables['PRECT'][time_indices_NDJFM,:,:]*86400.*1000

In [8]:
threshold=0.1

In [9]:
# load indices arrays
ccal_latlon_indices_array = numpy.load('../ccal_latlon_indices_array.npy')
ncal_latlon_indices_array = numpy.load('../ncal_latlon_indices_array.npy')
scal_latlon_indices_array = numpy.load('../scal_latlon_indices_array.npy')

# Do calculations for central California

In [15]:
# string together all daily precip values
latlon_indices = ccal_latlon_indices_array
PRECT_daily_perc = numpy.zeros((latlon_indices.shape))
perc_score=95

for latlon_combo in range(latlon_indices.shape[0]):
    print(latlon_combo)
    PRECT_data = numpy.empty((0))
    for file in range(file_list.size):
        ncfile = netCDF4.Dataset(working_dir + file_list[i])
        PRECT_time_var = ncfile.variables['time']
        PRECT_time_dates = netCDF4.num2date(PRECT_time_var[:], PRECT_time_var.units, PRECT_time_var.calendar)
        time_indices_NDJFM = numpy.array([t.month in [11,12,1,2,3] for t in PRECT_time_dates], dtype=bool)
        PRECT_data = numpy.append(PRECT_data, ncfile.variables['PRECT'][time_indices_NDJFM,latlon_indices[latlon_combo,0],latlon_indices[latlon_combo,1]]*86400.*1000.)
        ncfile.close()
    PRECT_data = PRECT_data[PRECT_data>threshold]
    PRECT_daily_perc[latlon_combo] = numpy.percentile(PRECT_data, q=perc_score)

0
1
2
3
4
5
6
7
8
